# How to use transfer learning and fine-tuning in Keras and Tensorflow to build an image recognition system and classify (almost) any object
Tutorial: https://deeplearningsandbox.com/how-to-use-transfer-learning-and-fine-tuning-in-keras-and-tensorflow-to-build-an-image-recognition-94b0b02444f2
Code: https://github.com/DeepLearningSandbox/DeepLearningSandbox/blob/master/transfer_learning/fine

In [21]:
import numpy as np
import os, random
from tqdm import tqdm

In [22]:
# Learning how np.random.choice works!
train_count = 0
test_count = 0
for i in range(10000):
    if np.random.choice(['train', 'test'], p=[0.9, 0.1]) == 'train':
        train_count += 1
    else:
        test_count += 1

print('Train count: ', train_count)
print('Test count: ', test_count)

Train count:  8984
Test count:  1016


In [26]:
# Split images using tutorial recommended structure
if not os.path.isdir(os.path.join(os.getcwd(), 'train_dir')):
    os.makedirs(os.path.join(os.getcwd(), 'train_dir', 'cat'), exist_ok=True)
    os.makedirs(os.path.join(os.getcwd(), 'train_dir', 'dog'), exist_ok=True)
    os.makedirs(os.path.join(os.getcwd(), 'val_dir', 'cat'), exist_ok=True)
    os.makedirs(os.path.join(os.getcwd(), 'val_dir', 'dog'), exist_ok=True)

    for _ in tqdm(os.listdir("train")):
        random_filename = random.choice(os.listdir("train"))
        random_filename_full = os.path.join(os.getcwd(), 'train', random_filename)
        class_str = random_filename.split('.')[0]
        dataset = np.random.choice(['train', 'val'], p=[0.9, 0.1])
        destination_filename_full = os.path.join(os.getcwd(), dataset + '_dir', class_str, random_filename)    
        os.rename(random_filename_full, destination_filename_full)

In [25]:
import sys
import glob
import argparse
import matplotlib.pyplot as plt

from keras import __version__
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

Using TensorFlow backend.


In [28]:
batch_size = 128
train_dir = os.path.join(os.getcwd(), 'train_dir')
val_dir = os.path.join(os.getcwd(), 'val_dir')

train_datagen =  ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=batch_size,
)
validation_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=batch_size,
)

Found 22534 images belonging to 2 classes.
Found 2466 images belonging to 2 classes.


In [29]:
base_model = InceptionV3(weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 49s       

In [31]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x) 
predictions = Dense(2, activation='softmax')(x) 
model = Model(inputs=base_model.input, outputs=predictions)

In [32]:
# Transfer learning
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Training
nb_train_samples = sum([len(files) for r, d, files in os.walk(train_dir)]) - 1
nb_val_samples = sum([len(files) for r, d, files in os.walk(val_dir)]) - 1

history = model.fit_generator(
    generator=train_generator, 
    steps_per_epoch=nb_train_samples//batch_size, 
    epochs=3, 
    verbose=1, 
    validation_data=validation_generator, 
    validation_steps=nb_val_samples, 
    class_weight='auto',
    workers=8)

#history = model.fit_generator(
#    train_generator,
#    samples_per_epoch=nb_train_samples,
#    nb_epoch=3,
#    validation_data=validation_generator,
#    nb_val_samples=nb_val_samples,
#    class_weight='auto')

model.save('inceptionv3-ft.model')

Epoch 1/3
 11/176 [>.............................] - ETA: 7357s - loss: 8.1163 - acc: 0.4964

In [ ]:
%matplotlib inline 

def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))
  
    plt.plot(epochs, acc, 'r.')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Training and validation accuracy')
  
    plt.figure()
    plt.plot(epochs, loss, 'r.')
    plt.plot(epochs, val_loss, 'r-')
    plt.title('Training and validation loss')
    plt.show()